# **Scri Tutorial: Asymptotic Bondi Data**

_A Python/numba code for manipulating time-dependent functions of spin-weighted spherical harmonics, now with more convenient support for working with asymptotic Bondi data!_

First make sure you are using the master branch of the `10220` fork. To do this, run the following in your terminal in a location where you would like to clone the scri directory:

```
conda update conda -y
conda update --all -y
conda install -c conda-forge scri
git clone git@github.com:10220/scri.git
cd scri
pip install .
```
By first installing the upstream version of `scri` through conda, we make sure that all the dependencies are satisfied. Then cloning and pip installing 10220's fork of `scri` will override conda installation of the upstream `scri`. NOTE: Make sure that the `pip` command here is your conda pip and not your system pip!

Next you'll probably want some waveforms to play with. You can find the Weyl Scalars Catalog on the wheeler cluster at `/home/dante/my_data/weyl_scalars_catalog_RPXM`. If you need help deciding which waveforms are right for you, feel free to slack `Dante Iozzo` or email him at `dai32@cornell.edu`.

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scri 
from spherical_functions import LM_index as lm # for easily accessing mode data

# There is a Numba bug that spits out some nasty warnings when using some scri functions.
# This is the band-aid until the Numba bug is fixed:
from numba.core.errors import NumbaWarning, NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
warnings.simplefilter('ignore', category=NumbaWarning)
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

The new kid on the block is the `AsymptoticBondiData` container, called ABD for short. While it still does not have all the features of the `WaveformModes` objects, it has most of the important ones and a bunch of new flash gadgets that we'll explore together in this notebook! This is stil in active development so please let me know if you find a bug.

You can use the function `scri.SpEC.file_io.create_abd_from_h5` to import our waveforms and create an ABD. For consistency with most of the literature that we use, our waveforms will be converted to the Moreschi-Boyle sign convention. See the Appendix B of arXiv:2010.15200 for more information on this sign convention.

In [ ]:
waveform_dir = "/Users/tim/OneDrive/All Documents/Code/NumericalRel/Ubuntu Directories/Jupyter/bianchi_identities/Data/Default" # Example: "/home/dante/weyl_scalar_catalog/bbh_q1_nospin/Lev5/bondi_cce_com"
cce_radius = 450 # The second smallest CCE radius is typically the optimal choice

abd = scri.SpEC.file_io.create_abd_from_h5(
    h    = f'{waveform_dir}/rh_FiniteRadii_CodeUnits.h5',
#     Psi4 = f'{waveform_dir}/rMPsi4_BondiCce_R{cce_radius:04}_CoM.h5',
#     Psi3 = f'{waveform_dir}/r2Psi3_BondiCce_R{cce_radius:04}_CoM.h5',
#     Psi2 = f'{waveform_dir}/r3Psi2OverM_BondiCce_R{cce_radius:04}_CoM.h5',
#     Psi1 = f'{waveform_dir}/r4Psi1OverM2_BondiCce_R{cce_radius:04}_CoM.h5',
#     Psi0 = f'{waveform_dir}/r5Psi0OverM3_BondiCce_R{cce_radius:04}_CoM.h5',
    file_format = 'RPXM',
)

# Or for Extrapolated waveforms
# abd = scri.SpEC.file_io.abd_from_h5(
#     h    = f'{waveform_dir}/rhOverM_Extrapolated_N5_CoM_Mem.h5',
#     Psi4 = f'{waveform_dir}/rMPsi4_Extrapolated_N7_CoM.h5',
#     Psi3 = f'{waveform_dir}/r2Psi3_Extrapolated_N7_CoM.h5',
#     Psi2 = f'{waveform_dir}/r3Psi2OverM_Extrapolated_N5_CoM.h5',
#     Psi1 = f'{waveform_dir}/r4Psi1OverM2_Extrapolated_N4_CoM.h5',
#     Psi0 = f'{waveform_dir}/r5Psi0OverM3_Extrapolated_N2_CoM.h5',
#     file_format = 'RPXM',
# )

All waveforms use the same time series, which can be access by:

In [ ]:
print(abd.t)

The data for any individual quantity can be accessed by:

In [ ]:
abd.psi4

The ABD stores the Newman-Penrose shear $\sigma$ instead of the gravitational wave strain $h$. In the Moreschi-Boyle convention they are related by $h = \bar{\sigma}$. This conversion is done automatically in the `create_abd_from_h5` function. So if you want to get the strain waveform you'll have to do `abd.sigma.bar`, but more info on that below. 

Individual modes can be access by either the `abd.psi4.index` function, or with the `spherical_functions.LM_index` function which I have aliased to `lm`. The third argument of `lm` is `ell_min` but `ell_min=0` for `abd` so you can always just set it to zero.

In [ ]:
# Access the (2,1) mode of psi4
abd.psi4[:, abd.psi4.index(2,1)]
abd.psi4[:, lm(2,1,0)]

You'll notice `abd.psi4` is not a `WaveformModes` object but a `ModesTimeSeries`. This allows us to perform operations more freely on the data and even compose operations:

In [ ]:
# take a derivative or two
psi4_dot  = abd.psi4.dot
psi4_ddot = abd.psi4.ddot

# Integrate once or twice
psi4_int  = abd.psi4.int
psi4_iint = abd.psi4.iint

# Apply the eth or ethbar operator (using the GHP definition, which is the one that is natural to the Moreschi-Boyle convention)
eth_psi4    = abd.psi4.eth_GHP
ethbar_psi4 = abd.psi4.ethbar_GHP
# If you really want to use the Newman-Penrose operators then you can do:
# eth_psi4    = abd.psi4.eth
# ethbar_psi4 = abd.psi4.ethbar

# Get fancy and combine them together
abd.sigma.dot.eth_GHP.eth_GHP

# Don't believe it? Check the spin-weight
abd.sigma.dot.eth_GHP.eth_GHP.s

We can add `abd` quantities together, but only in a way that makes sense. It makes no sense to sum two quantities of different spin weights. So naturally only one of the following will work:

In [ ]:
abd.psi4 + abd.psi3 # This will throw an error
abd.psi4.eth_GHP + abd.psi3 # This will work

# Check the spin weight on the whole thing:
(abd.psi4.eth_GHP + abd.psi3).s

We can also multiply quantites together! No longer will you have to convert to `WaveformGrid` objects and back again to compute this product! This does take a while though so if you're going to use this more than once, be sure to store the result as a variable:

In [22]:
sigma_psi4 = abd.sigma * abd.psi4

For a faster version you can do the following instead. Instead of multiplying mode weights, this will convert the quantities to values on a spherical grid, perfom the product, then transform back to a mode representation. Mike is working on a faster way to compute Wigner-D matrices, but until then you can use this function:

In [ ]:
sigma_psi4 = abd.sigma.grid_multiply(abd.psi4)

Now it's super simple to compute a Bondi balance law. Let's check $\dot{\psi}_2 - \eth\psi_3 - \sigma\psi_4 = 0$:

In [ ]:
bondi_balance_law_psi2 = abd.psi2.dot - abd.psi3.eth_GHP - sigma_psi4

But since we are going to be doing this a lot, we already have built-in tools for checking these constraints:

In [ ]:
# Read the help text on these functions:

# help(abd.bondi_constraints)
# help(abd.bondi_violations)
# help(abd.bondi_violation_norms)
# help(abd.bondi_four_momentum)

These functions take a while to run though, so I find it best to do the following:

In [ ]:
# This takes the longest so let's only do it once
constraints = abd.bondi_constraints()

# This is just abd.bondi_violations
violations = [lhs-rhs for (lhs, rhs) in constraints]

# This is just abd.bondi_violation_norms
violation_norms = np.array([v.norm() for v in violations])

# Sometimes it's nice to see the relative norms (i.e. normalized with respect to the RHS of the balance laws)
normalization = np.array([v.norm() for v in [rhs for (lhs, rhs) in constraints]])
normalized_violation_norms = violation_norms / normalization

We can make quick plots of the violations:

In [ ]:
# If you have LaTeX support for matplotlib, then uncomment the following for a nicer legend
# legend_labels = [
#     r"$\dot{\psi}_0 = \eth \psi_1 + 3 \sigma \psi_2$",
#     r"$\dot{\psi}_1 = \eth \psi_2 + 2 \sigma \psi_3$",
#     r"$\dot{\psi}_2 = \eth \psi_3 + \sigma \psi_4$",
#     r"$\psi_3 = - \eth \dot{\bar{\sigma}} $",
#     r"$\psi_4 = - \ddot{\bar{\sigma}}$",
#     r"$\mathrm{Im}[\psi_2] = -\mathrm{Im}[\eth^2 \bar{\sigma} + \sigma \dot{\bar{\sigma}}]$"
# ]

legend_labels = [
    "Psi0_dot Eq.",
    "Psi1_dot Eq.",
    "Psi2_dot Eq.",
    "Psi3 Eq.",
    "Psi4 Eq.",
    "Im[Psi2] Eq."
]

for i in reversed(range(6)):
    plt.semilogy(abd.t, violation_norms[i], label=legend_labels[i])
plt.xlabel(r"u [M]")
plt.title(f"Bondi Violation Norms")
plt.legend(ncol=2, prop={'size': 8})
plt.show()

A BMS transformation on an ABD will transform all the data together. Read the help text for more info.

In [ ]:
transformed_abd = abd.transform(boost_velocity=[0,0,1e-4])
# help(abd.transform)

### CAVEATS:

When taking the real part, imaginary part, or the complex conjugate, be very careful to know whether you are acting on the whole quantity or just the modes of the quantity. For example:

In [ ]:
# This takes the real part of the quantity Psi2, and then returns the modes of Re(Psi2). The mode weights are still complex!
abd.psi2.real

# This returns the real part of the mode weights of Psi2. This is an array of real numbers!
abd.psi2.ndarray.real

# This returns the modes of the complex conjugate of sigma. This is usually what you want.
abd.sigma.bar 

# This returns the complex conjguate of the modes of sigma. This is usually NOT what you want.
np.conjugate(abd.sigma.ndarray)

# So the balance law for Psi3 should be:
abd.psi3 + abd.sigma.bar.dot.eth_GHP

Sometimes this can be a pain for plotting because you'll have to convert to a numpy array to plot the real, imag, abs of a mode:

In [ ]:
plt.plot(abd.t, abd.sigma[:,lm(2,2,0)].ndarray.real)
plt.plot(abd.t, abd.sigma[:,lm(2,2,0)].ndarray.imag)
plt.plot(abd.t, np.abs(abd.sigma[:,lm(2,2,0)].ndarray))
plt.show()

# New Features!

### Tools to make life easier

You can make a copy of an ABD with:

In [ ]:
new_abd = abd.copy()

You can also interpolate all the waveforms in an ABD to a new set of times. This will even interpolate the frame rotors to the new times.

In [ ]:
interpolated_abd = abd.interpolate(new_t)

### Rotating frames

I hope you don't get vertigo, because ABD now supports frame rotations! Given a rotor `R` or an array of rotors `R`, you can rotate an individual value:

In [ ]:
rotated_psi4 = abd.psi4.rotate_decomposition_basis(R)
# also:
rotated_psi4 = abd.psi4.rotate_physical_system(R)

This will just return the rotated quantity and not store any more information about the rotation. However, you can also rotate all of the objects in an ABD and the ABD will then store the rotor or array of rotors that made the transformation:

In [ ]:
abd.rotate_decomposition_basis(R)
abd.rotate_physical_system(R)

You can even go to the corotating or coprecessing frame. However, this is different for each quantity so you have to specify which quantity you will use to set the corotating (coprecessing) frame:

In [ ]:
abd.to_corotating_frame(scri.psi4)
abd.to_coprecessing_frame(scri.psi4)

At any point you can undo all your frame rotations by:

In [ ]:
abd.to_inertial_frame()

The rotors or array of rotors that define the frame can be accessed by:

In [ ]:
abd.frame

### Supermomentum

Get the Bondi 4-momentum or the mass aspect from the data:

In [ ]:
P = abd.bondi_four_momentum()
M = abd.mass_aspect()

Or quickly compute the supermomentum. There are four definitions of the supermomentum currently available to be computed so DEFINITELY see the help text:

In [ ]:
Psi = abd.supermomentum('Moreschi')
# help(abd.supermomentum)

For the sake of testing, Moreschi provides an equation for how the Moreschi-supermomentum transforms under a BMS transformation. This is available in the function:

In [ ]:
help(scri.asymptotic_bondi_data.supermomentum.transform_moreschi_supermomentum)

# So in a perfect world we should find:

β = # some boost vector
α = # some supertranslation
Ψ = abd.supermomentum('Moreschi')
abd_prime = abd.transform(boost_velocity=β, supertranslation=α)
Ψ_prime_0 = abd_prime.supermomentum('Moreschi')
Ψ_prime_1 = scri.asymptotic_bondi_data.supermomentum.transform_moreschi_supermomentum(Ψ, boost_velocity=β, supertranslation=α)

# We should get Ψ_prime_0 == Ψ_prime_1, but there are issues around merger.

We can also provide another analysis of the spacetime by computing the speciality index $S$,
$$S = \frac{27 J^2}{I^3}$$
where the curavture invariants $I$ and $J$ are defined by,
\begin{align}
I &= \psi_4 \psi_0 - 4 \psi_3 \psi_1 + 3 \psi_2^2 \\
J &= \mathrm{det} \begin{pmatrix} \psi_4 & \psi_3 & \psi_2 \\ \psi_3 & \psi_2 & \psi_1 \\ \psi_2 & \psi_1 & \psi_0 \end{pmatrix}
\end{align}
For algebraically special spacetimes, we get $S=1$. The following function will give the modal decomposition of $S$:

In [ ]:
S = abd.speciality_index()